# Notes for the Project Report

The best performing model is LR tuned hyperparameters test accuracy:  0.9805518203097764
Confusion matrix:  
 [[19345   734]
 [   47 20032]] 
 
All other algorythms accuracy is ~0.5.

# LOADING TRAIN AND TEST DATA

In [1]:
# Train data.
import pandas as pd
data_train = pd.read_csv("Standard_FS_RFElog10_train_output.csv")
data_train.shape

(97044, 11)

In [2]:
data_train.head()

,class,frame.cap_len,radiotap.channel.type.cck,wlan.fc.ds,wlan.fc.protected,wlan.fc.retry,wlan.seq,wlan_mgt.fixed.beacon,wlan_mgt.fixed.timestamp,wlan_mgt.tagged.all,wlan_mgt.tim.dtim_period
0,0,-0.521063,-1.533249,-0.412052,-1.542391,-0.285459,2.653966,-0.293653,-0.150600,-0.314605,-0.182614
1,0,-0.546879,-1.533249,-1.523053,-1.542391,-0.285459,-0.831461,-0.293653,-0.150600,-0.314605,-0.182614
2,0,-0.347726,0.652210,-1.523053,-1.542391,-0.285459,0.109363,3.405374,2.906133,3.178590,7.769600
3,0,-0.279498,0.652210,-1.523053,-1.542391,-0.285459,-0.563420,3.405374,10.555249,3.178590,3.793543
4,0,-0.347726,0.652210,-1.523053,-1.542391,-0.285459,0.114777,3.405374,2.906197,3.178590,7.769600


In [3]:
# split values into inpits and outputs.
values_train = data_train.values
X_train = values_train[:,1:11]
y_train = values_train[:,0]

data_train.shape

(97044, 11)

In [4]:
# Test data.
data_test = pd.read_csv("Standard_FS_RFElog10_test_output.csv")

# split values into inpits and outputs.
values_test = data_test.values
X_test = values_test[:,1:11]
y_test = values_test[:,0]

data_test.shape

(40158, 11)

# LOGISTIC REGRESSION

## LR with default hyperparameters

In [5]:
# Initiate the LR model with defualt hyperparameters.
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [6]:
# Fit the model using default hyperparameters.
# K, you don't split into train and validate sets??
lr.fit(X_train, y_train)

/Users/konstantinorlovskiy/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [7]:
# Run predictions on TEST set and see the accuracy.
lr_score = lr.score(X_test,y_test)
print(lr_score)

0.9800039842621644


In [8]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
lr_cm = confusion_matrix(y_test, lr.predict(X_test))
print(lr_cm)

[[19323   756]
 [   47 20032]]


## LR hyperparameters tuning (Random Search)

In [9]:
from sklearn.model_selection import RandomizedSearchCV

# Create array of values for tuned hyperparameters.
lr_params = {'C' : [0.01, 0.05, 0.1, 0.5, 1.0, 1.5, 2.0], 
             'max_iter' : [5, 10, 50, 100, 150, 200, 300]
             }

In [10]:
# Run random search and initiate the model with tuned parameters.
lr_random = RandomizedSearchCV(estimator=lr, param_distributions=lr_params, cv = 3, n_jobs=-1, random_state = 2019)

import time
start_time = time.time()
lr_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (lr_random.best_score_, lr_random.best_params_))
print("Execution time: " + str((finish_time - start_time)))

/Users/konstantinorlovskiy/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Best: 0.981946 using {'max_iter': 150, 'C': 1.5}
Execution time: 2.718111038208008


In [11]:
# Apply best values of hyperparameters to the model.
lr_tuned = lr_random.best_estimator_

In [12]:
# Train the tuned model on TRAIN set and check the accuracy
lr_tuned.fit(X_train, y_train)
lr_tuned_score = lr_tuned.score(X_test,y_test)
print(lr_tuned_score)

/Users/konstantinorlovskiy/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9805518203097764


In [13]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
lr_tuned_cm = confusion_matrix(y_test, lr_tuned.predict(X_test))
print(lr_tuned_cm)

[[19345   734]
 [   47 20032]]


## LR tuning Results

In [14]:
print("LR default hyperparameters test accuracy: ", lr_score,', parameters: ', '\n', lr.get_params())
print('Confusion matrix: ', '\n', lr_cm)
print()
print("LR tuned hyperparameters test accuracy: ", lr_tuned_score,', parameters: ', '\n', lr_tuned.get_params())
print('Confusion matrix: ', '\n', lr_tuned_cm)

LR default hyperparameters test accuracy:  0.9800039842621644 , parameters:  
 {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'warn', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'warn', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Confusion matrix:  
 [[19323   756]
 [   47 20032]]

LR tuned hyperparameters test accuracy:  0.9805518203097764 , parameters:  
 {'C': 1.5, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 150, 'multi_class': 'warn', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'warn', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Confusion matrix:  
 [[19345   734]
 [   47 20032]]


# DECISION TREE

## DT with default hyperparameters

In [15]:
# Initiate a DT model using default hyperparameters.
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()

In [16]:
# Train model on train data.
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [17]:
# Check model accuracy on the TEST set.
dt_score = dt.score(X_test, y_test)
print(dt_score)

0.5330444743264107


In [18]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
dt_cm = confusion_matrix(y_test, dt.predict(X_test))
print(dt_cm)

[[19939   140]
 [18612  1467]]


## DT hyperparameters tuning (Random Search)

In [19]:
# https://medium.com/@mohtedibf/indepth-parameter-tuning-for-decision-tree-6753118a03c3
# Create array of values for tuned hyperparameters.
dt_params = {'max_depth': [None, 1, 3, 5, 10, 50, 100, 300], 
             'min_samples_split': [2, 5, 10, 50, 100], 
             'min_samples_leaf': [1, 2, 5, 10, 50, 100]
            }

In [20]:
# Run random search.
from sklearn.model_selection import RandomizedSearchCV
dt_random = RandomizedSearchCV(estimator=dt, param_distributions=dt_params, cv = 10, n_jobs=-1, random_state = 2019)

import time
start_time = time.time()
dt_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (dt_random.best_score_, dt_random.best_params_))
print("Execution time: " + str((finish_time - start_time)))

Best: 0.999299 using {'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 10}
Execution time: 1.4584219455718994


In [21]:
# Apply best values of hyperparameters to the model.
dt_tuned = dt_random.best_estimator_

In [22]:
# Train the tuned model on TRAIN set and check the accuracy
dt_tuned.fit(X_train, y_train)
dt_tuned_score = dt_tuned.score(X_test,y_test)
print(dt_tuned_score)

0.5332934907116889


In [23]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
dt_tuned_cm = confusion_matrix(y_test, dt_tuned.predict(X_test))
print(dt_tuned_cm)

[[19949   130]
 [18612  1467]]


## DT tuning Results

In [24]:
print("DT default hyperparameters test accuracy: ", dt_score,', parameters: ', '\n', dt.get_params())
print('Confusion matrix: ', '\n', dt_cm)
print()
print("DT tuned hyperparameters test accuracy: ", dt_tuned_score,', parameters: ', '\n', dt_tuned.get_params())
print('Confusion matrix: ', '\n', dt_tuned_cm)

DT default hyperparameters test accuracy:  0.5330444743264107 , parameters:  
 {'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': False, 'random_state': None, 'splitter': 'best'}
Confusion matrix:  
 [[19939   140]
 [18612  1467]]

DT tuned hyperparameters test accuracy:  0.5332934907116889 , parameters:  
 {'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0.0, 'presort': False, 'random_state': None, 'splitter': 'best'}
Confusion matrix:  
 [[19949   130]
 [18612  1467]]


# RANDOM FOREST

## RF with default hyperparameters

In [25]:
# Initiate a RF model using default hyperparameters.
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [26]:
# Train model on train data.
rf.fit(X_train, y_train)

/Users/konstantinorlovskiy/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [27]:
# Check model accuracy on the TEST set.
rf_score = rf.score(X_test, y_test)
print(rf_score)

0.5347626873848299


In [28]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
rf_cm = confusion_matrix(y_test, rf.predict(X_test))
print(rf_cm)

[[20007    72]
 [18611  1468]]


## RF hyperparameters tuning (Random Search)

In [29]:
# Define a grid of hyperparameters.
rf_params = { 'n_estimators': [1, 5, 10, 30, 50, 100, 200, 500], 
             'max_depth': [None, 1, 2, 4, 8, 20, 50, 100], 
             'min_samples_leaf': [1, 5, 10, 50, 100], 
             'max_features': [None, 'auto', 'log2']
            }

In [30]:
# Run random search.
from sklearn.model_selection import RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=rf_params, cv = 3, n_jobs=-1, random_state = 2019)

import time
start_time = time.time()
rf_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (rf_random.best_score_, rf_random.best_params_))
print("Execution time: " + str((finish_time - start_time)))

Best: 0.995590 using {'n_estimators': 1, 'min_samples_leaf': 5, 'max_features': 'log2', 'max_depth': 8}
Execution time: 7.109869956970215


In [31]:
# Apply best values of hyperparameters to the model.
rf_tuned = rf_random.best_estimator_

In [32]:
# Train the tuned model on TRAIN set and check the accuracy
rf_tuned.fit(X_train, y_train)
rf_tuned_score = rf_tuned.score(X_test,y_test)
print(rf_tuned_score)

0.5266945565018178


In [33]:
# Build confusion matrix.
rf_tuned_cm = confusion_matrix(y_test, rf_tuned.predict(X_test))
print(rf_tuned_cm)

[[19733   346]
 [18661  1418]]


## RF tuning Results

In [34]:
print("RF default hyperparameters test accuracy: ", rf_score,', parameters: ', '\n', rf.get_params())
print('Confusion matrix: ', '\n', rf_cm)
print()
print("RF tuned hyperparameters test accuracy: ", rf_tuned_score,', parameters: ', '\n', rf_tuned.get_params())
print('Confusion matrix: ', '\n', rf_tuned_cm)

RF default hyperparameters test accuracy:  0.5347626873848299 , parameters:  
 {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 10, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
Confusion matrix:  
 [[20007    72]
 [18611  1468]]

RF tuned hyperparameters test accuracy:  0.5266945565018178 , parameters:  
 {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 8, 'max_features': 'log2', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 5, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
Confusion matrix:  
 [[19733   34

# SVM (SVC)

## SVC with default hyperparameters

In [35]:
from sklearn import svm
svc = svm.SVC()

In [36]:
svc.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [37]:
svc_score = svc.score(X_test, y_test)
print(svc_score)

0.5217889337118382


In [38]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
svc_cm = confusion_matrix(y_test, svc.predict(X_test))
print(svc_cm)

[[19510   569]
 [18635  1444]]


 ## SVC hyperparameters tuning (Random Search)

In [39]:
# Define a grid of hyperparameters.
svc_params = { 'C': [0.1, 0.5, 1, 3, 5], 
             'gamma': ['scale', 'auto', 0.01, 0.1, 1, 10]
            }

In [40]:
# Run random search.
from sklearn.model_selection import RandomizedSearchCV
svc_random = RandomizedSearchCV(estimator=svc, n_iter=3, param_distributions=svc_params, cv = 3, n_jobs=-1, 
                                random_state = 2019)

import time
start_time = time.time()
svc_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (svc_random.best_score_, svc_random.best_params_))
print("Execution time: " + str((finish_time - start_time)))

Best: 0.993694 using {'gamma': 'auto', 'C': 3}
Execution time: 35.372045040130615


In [41]:
# Apply best values of hyperparameters to the model.
svc_tuned = svc_random.best_estimator_

In [42]:
# Train the tuned model on TRAIN set and check the accuracy
svc_tuned.fit(X_train, y_train)
svc_tuned_score = svc_tuned.score(X_test,y_test)
print(svc_tuned_score)

0.5226604910603118


In [43]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
svc_tuned_cm = confusion_matrix(y_test, svc_tuned.predict(X_test))
print(svc_tuned_cm)

[[19545   534]
 [18635  1444]]


## SVC tuning Results

In [44]:
print("SVC default hyperparameters test accuracy: ", svc_score, 
      ', parameters: ', '\n', svc.get_params())
print('Confusion matrix: ', '\n', svc_cm)
print()
print("SVC tuned hyperparameters test accuracy: ", svc_tuned_score, 
      ', parameters: ', '\n', svc_tuned.get_params())
print('Confusion matrix: ', '\n', svc_tuned_cm)

SVC default hyperparameters test accuracy:  0.5217889337118382 , parameters:  
 {'C': 1.0, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'auto_deprecated', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Confusion matrix:  
 [[19510   569]
 [18635  1444]]

SVC tuned hyperparameters test accuracy:  0.5226604910603118 , parameters:  
 {'C': 3, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Confusion matrix:  
 [[19545   534]
 [18635  1444]]


# KNN classifier

## KNN with default hyperparameters

In [45]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [46]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [47]:
knn_score = knn.score(X_test, y_test)
print(knn_score)

0.5350117037701081


In [48]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
knn_cm = confusion_matrix(y_test, knn.predict(X_test))
print(knn_cm)

[[20041    38]
 [18635  1444]]


## KNN hyperparameters tuning (Random Search)¶

In [49]:
# Define a grid of hyperparameters.
knn_params = {'n_neighbors': [3, 5, 10, 20, 50, 100], 
              'weights': ['uniform', 'distance'], 
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 
              'leaf_size': [10, 30, 50, 100], 
              'p': [1, 2]
            }

In [50]:
# Run random search.
from sklearn.model_selection import RandomizedSearchCV
knn_random = RandomizedSearchCV(estimator=knn, n_iter=3, param_distributions=knn_params, cv = 3, n_jobs=-1, 
                                random_state = 2019)

import time
start_time = time.time()
knn_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (knn_random.best_score_, knn_random.best_params_))
print("Execution time: " + str((finish_time - start_time)))

Best: 0.994405 using {'weights': 'uniform', 'p': 2, 'n_neighbors': 10, 'leaf_size': 100, 'algorithm': 'brute'}
Execution time: 60.71341013908386


In [51]:
# Apply best values of hyperparameters to the model.
knn_tuned = knn_random.best_estimator_

In [52]:
# Train the tuned model on TRAIN set and check the accuracy
knn_tuned.fit(X_train, y_train)
knn_tuned_score = knn_tuned.score(X_test,y_test)
print(knn_tuned_score)

0.5347626873848299


In [53]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
knn_tuned_cm = confusion_matrix(y_test, knn_tuned.predict(X_test))
print(knn_tuned_cm)

[[20031    48]
 [18635  1444]]


## KNN tuning Results

In [54]:
print("KNN default hyperparameters test accuracy: ", knn_score, 
      ', parameters: ', '\n', knn.get_params())
print('Confusion matrix: ', '\n', knn_cm)
print()
print("KNN tuned hyperparameters test accuracy: ", knn_tuned_score, 
      ', parameters: ', '\n', knn_tuned.get_params())
print('Confusion matrix: ', '\n', knn_tuned_cm)

KNN default hyperparameters test accuracy:  0.5350117037701081 , parameters:  
 {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
Confusion matrix:  
 [[20041    38]
 [18635  1444]]

KNN tuned hyperparameters test accuracy:  0.5347626873848299 , parameters:  
 {'algorithm': 'brute', 'leaf_size': 100, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 10, 'p': 2, 'weights': 'uniform'}
Confusion matrix:  
 [[20031    48]
 [18635  1444]]


# Naive Bayes Classifier (NBC)

In [55]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
nbc = GaussianNB()

In [56]:
# Fit the model and display score
nbc.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [57]:
# Run predictions on TEST set and see the accuracy.
nbc_score = nbc.score(X_test,y_test)
print(nbc_score)

0.5


In [58]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
nbc_cm = confusion_matrix(y_test, nbc.predict(X_test))
print(nbc_cm)

[[20079     0]
 [20079     0]]


## NBC hyperparameters tuning (Random Search)

In [59]:
# Define a grid of hyperparameters.
nbc_params = {'var_smoothing': [1e-12, 1e-10, 1e-09, 1e-05, 1e-04, 1e-03, 1e-02, 1e-01, 1]
            }

In [60]:
# Run random search.
from sklearn.model_selection import RandomizedSearchCV
nbc_random = RandomizedSearchCV(estimator=nbc, n_iter=3, param_distributions=nbc_params, cv = 3, n_jobs=-1, 
                                random_state = 2019)

import time
start_time = time.time()
nbc_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (nbc_random.best_score_, nbc_random.best_params_))
print("Execution time: " + str((finish_time - start_time)))

Best: 0.928991 using {'var_smoothing': 0.0001}
Execution time: 0.1779179573059082


In [61]:
# Apply best values of hyperparameters to the model.
nbc_tuned = nbc_random.best_estimator_

In [62]:
# Train the tuned model on TRAIN set and check the accuracy
nbc_tuned.fit(X_train, y_train)
nbc_tuned_score = nbc_tuned.score(X_test,y_test)
print(nbc_tuned_score)

0.5


In [63]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
nbc_tuned_cm = confusion_matrix(y_test, nbc_tuned.predict(X_test))
print(nbc_tuned_cm)

[[20079     0]
 [20079     0]]


## NBC tuning Results

In [64]:
print("NBC default hyperparameters test accuracy: ", nbc_score, 
      ', parameters: ', '\n', nbc.get_params())
print('Confusion matrix: ', '\n', nbc_cm)
print()
print("NBC tuned hyperparameters test accuracy: ", nbc_tuned_score, 
      ', parameters: ', '\n', nbc_tuned.get_params())
print('Confusion matrix: ', '\n', nbc_tuned_cm)

NBC default hyperparameters test accuracy:  0.5 , parameters:  
 {'priors': None, 'var_smoothing': 1e-09}
Confusion matrix:  
 [[20079     0]
 [20079     0]]

NBC tuned hyperparameters test accuracy:  0.5 , parameters:  
 {'priors': None, 'var_smoothing': 0.0001}
Confusion matrix:  
 [[20079     0]
 [20079     0]]


# Compare Algorithms Performance

In [65]:
print("LR tuned hyperparameters test accuracy: ", lr_tuned_score)
print('Confusion matrix: ', '\n', lr_tuned_cm, '\n')

print("DT tuned hyperparameters test accuracy: ", dt_tuned_score)
print('Confusion matrix: ', '\n', dt_tuned_cm, '\n')

print("RF tuned hyperparameters test accuracy: ", rf_tuned_score)
print('Confusion matrix: ', '\n', rf_tuned_cm, '\n')

print("SVC tuned hyperparameters test accuracy: ", svc_tuned_score)
print('Confusion matrix: ', '\n', svc_tuned_cm)

print("KNN tuned hyperparameters test accuracy: ", knn_tuned_score)
print('Confusion matrix: ', '\n', knn_tuned_cm)

print("NBC tuned hyperparameters test accuracy: ", nbc_tuned_score)
print('Confusion matrix: ', '\n', nbc_tuned_cm)

LR tuned hyperparameters test accuracy:  0.9805518203097764
Confusion matrix:  
 [[19345   734]
 [   47 20032]] 

DT tuned hyperparameters test accuracy:  0.5332934907116889
Confusion matrix:  
 [[19949   130]
 [18612  1467]] 

RF tuned hyperparameters test accuracy:  0.5266945565018178
Confusion matrix:  
 [[19733   346]
 [18661  1418]] 

SVC tuned hyperparameters test accuracy:  0.5226604910603118
Confusion matrix:  
 [[19545   534]
 [18635  1444]]
KNN tuned hyperparameters test accuracy:  0.5347626873848299
Confusion matrix:  
 [[20031    48]
 [18635  1444]]
NBC tuned hyperparameters test accuracy:  0.5
Confusion matrix:  
 [[20079     0]
 [20079     0]]
